In [94]:
import pandas as pd
import numpy as np
from sklearn import decomposition
from dateutil.relativedelta import relativedelta
from dateutil.relativedelta import relativedelta
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [89]:
df = pd.read_csv('appl_data.csv')
result = pd.read_csv('is_default.csv')
on_site = pd.read_csv('behav_on_site.csv')

In [90]:
#Making one df
cnt=0
c = list(set(on_site['id_ref'].values))
clients = list(set(on_site['client_id'].values))
c.append('num_of_devices')
new_onsite = pd.DataFrame(columns = c)

for client in clients:
    num_of_devices = on_site.loc[on_site['client_id'] == client].device_id.shape[0]
    links = list(set(on_site.loc[on_site['client_id'] == client].id_ref.values))
    row = [0] * len(c)
    for i in range(len(c)-1):
        if c[i] in links:
            row[i] = 1
    row[-1] = num_of_devices
    new_onsite.loc[cnt] = row
    cnt+=1

val = new_onsite.values
pca = decomposition.PCA(n_components=20)
val = pca.fit_transform(val)
new_onsite = pd.DataFrame(val, columns = [i for i in range(20)])
new_onsite['client_id'] = clients

df = df.merge(new_onsite, left_on='client_id', right_on='client_id')
df = df.merge(result, left_on='appl_id', right_on='appl_id')

In [91]:
del_col = ['is_same_reg_lived_since',
           'region_reg',
           'empl_worker_count',
           'appl_id',
           'app_crtime',
           'client_id',
           'pass_bdate', 
           'birth',
           'lived_since',
           'jobsworksince']

df['birth'] = pd.to_datetime(df['birth'])
df['pass_bdate'] = pd.to_datetime(df['pass_bdate'])
df['lived_since'] = pd.to_datetime(df['lived_since'])
df['jobsworksince'] = pd.to_datetime(df['jobsworksince'])

def f1(end):
    r = relativedelta(pd.to_datetime('now'), end) 
    return '{}'.format(r.months)

def f2(end):
    r = relativedelta(pd.to_datetime('now'), end) 
    return '{}'.format(r.years)

df['jobsworksince'] = df['jobsworksince'].fillna(pd.to_datetime('now'))

df['age'] = df['birth'].apply(f1)
df['live_on_current_place_months'] = df['lived_since'].apply(f2)
df['work_on_current_place_months'] = df['jobsworksince'].apply(f2)

df = df.drop(del_col, axis=1)

df['gender'] = df['gender'].replace(2,0)
df = df.fillna(0)

In [98]:
to_one_hot = ['fam_status', 'region', 'empl_state', 'empl_type', 'education_area', 'education']
categ_col = to_one_hot + ['gender', 'df']

x = df.drop(categ_col, axis=1).values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df[df.drop(categ_col, axis=1).columns] = x_scaled

df = pd.get_dummies(df, columns = to_one_hot)

df['df'] = df['df'].replace('good',1)
df['df'] = df['df'].replace('bad',0)

data = df.loc[(df['df']==1) | (df['df']==0)]
to_pred = df.loc[(df['df']!=1) & (df['df']!=0)]
to_pred = to_pred.drop(['df'], axis=1)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [116]:
#LogReg

X = data.drop(['df'], axis=1).values
y = data['df'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)

pred = clf.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.717428167206587

# Other algorithms

In [133]:
def run_classifier(clf):
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
    return metrics.auc(fpr, tpr)

classifiers = [KNeighborsClassifier(n_neighbors = 2),
              SVC(),
              GaussianNB(),
              Perceptron(),
              LinearSVC(),
              SGDClassifier(),
              DecisionTreeClassifier(),
              RandomForestClassifier(n_estimators=100)]

result = []

for c in classifiers:
    score = run_classifier(c)
    result.append(score)
    
for score in result:
    print(score)

E:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and defa

0.545981451235814
0.7065682385276715
0.6561464765331303
0.7203900750726147
0.7207196094567365
0.7154500605346861
0.6654184576470286
0.7090163368621724
